# Implementações dos problemas de PLI - A2
### Disciplina: Programação Linear Inteira
### Integrantes: Daniel Jacob, Juliana Carrica, Wendell Oliveira
### Professor: Vincent Gerard Y. Guigues
### Monitor: Danilo

In [38]:
#Bibliotecas 
using JuMP
using GLPK

import Pkg; Pkg.add("MathOptInterface")


# Problema 1.1

In [14]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, q[1:2, 1:3] >= 0, Int)

# Função objetivo
c = [1 2 1.5; 2 3.5 2]  # Custos de transporte organizados por usina
@objective(model, Min, sum(q[i, j] * c[i, j] for i in 1:2, j in 1:3))

# Restrições de capacidade das usinas
@constraint(model, sum(q[1, j] for j in 1:3) <= 400)
@constraint(model, sum(q[2, j] for j in 1:3) <= 300)

# Restrições de demanda das fábricas
@constraint(model, sum(q[i,1] for i in 1:2) == 100)
@constraint(model, sum(q[i,2] for i in 1:2) == 200)
@constraint(model, sum(q[i,3] for i in 1:2) == 300)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Custo mínimo: ", objective_value(model))
println("Quantidades ótimas de transporte:")
for i in 1:2
    for j in 1:3
        println("q[$i, $j] = ", value(q[i, j]))
    end
end

# Imprimindo excedente no estoque
println("Excedente na usina 1: ", 400 -  value(sum(q[1, j] for j in 1:3)) )
println("Excedente na usina 2: ", 300 -  value(sum(q[2, j] for j in 1:3)) )

Custo mínimo: 1050.0
Quantidades ótimas de transporte:
q[1, 1] = 100.0
q[1, 2] = 200.0
q[1, 3] = 100.0
q[2, 1] = 0.0
q[2, 2] = 0.0
q[2, 3] = 200.0
Excedente na usina 1: 0.0
Excedente na usina 2: 100.0


# Problema 1.2

In [63]:
# Definindo o modelo
model_first = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model_first, T[1:2] >= 0, Int)

# Função objetivo
c = [7 9]  # Lucro por kilo de produto
@objective(model_first, Max, sum(T[j] * c[j] for j in 1:2))

# Restrições de limitação de matéria-prima
@constraint(model_first, T[1] + T[2] <= 8)
@constraint(model_first, 2T[1] + 3T[2] <= 19)
@constraint(model_first, T[2] <= 4)

# Resolvendo o modelo]
optimize!(model_first)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model_first))
println("Quantidades ótimas de produção:")
for i in 1:2
    println("T[$i] = ", value(T[i]))
end

#Exibindo o excedente de matéria-prima
println("Excedente Nitrato: ", 8 - ( value(T[1]) + value(T[2]) ))
println("Excedente Sal de Potássio: ", 19 - ( 2value(T[1]) + 3value(T[2])))
println("Excedente Fosfato: ", 4 -  (value(T[2]) ))




Lucro máximo: 62.0
Quantidades ótimas de produção:
T[1] = 5.0
T[2] = 3.0
Excedente Nitrato: 0.0
Excedente Sal de Potássio: 0.0
Excedente Fosfato: 1.0


In [64]:
# Definindo o modelo
model_sec = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model_sec, P[1:3] >= 0, Int)

# Função objetivo
c = [8, 19, 4]  # Quantidade de nutrientes por quilo de produto disponível
@objective(model_sec, Min, sum(P[i] * c[i] for i in 1:3))

# Restrições de lucro por quilo de produto
@constraint(model_sec, P[1] + 2* P[2] >= 700)
@constraint(model_sec, P[1] +3P[2] + P[3] >= 900)

# Resolvendo o modelo
optimize!(model_sec)

# Imprimindo o resultado
println("Minimização do preço à granel: ", objective_value(model_sec))
println("Preço por granel:")
for i in 1:3
    println("P[$i] = ", value(P[i]))
end


Minimização do preço à granel: 6200.0
Preço por granel:
P[1] = 300.0
P[2] = 200.0
P[3] = 0.0


# Problema 1.3

In [66]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, q[1:2 , 1:3] >= 0, Int)

# Função objetivo
c = [38 27 48 ; 37 58 45]  # Custo de transporte por rota
@objective(model, Max, sum(q[i,j] * c[i,j] for i in 1:2, j in 1:3))

# Restrições de limitação de matéria-prima
@constraint(model, sum(q[1,j] for j in 1:3) <= 6)
@constraint(model, sum(q[2,j] for j in 1:3) <= 6)

# Restrições de demanda das fábricas
@constraint(model, sum(q[i,1] for i in 1:2) == 5)
@constraint(model, sum(q[i,2] for i in 1:2) == 4)
@constraint(model, sum(q[i,3] for i in 1:2) == 3)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Custo mínimo de distribuição: ", objective_value(model))
println("Quantidades ótimas de distribuição:")
for i in 1:2
    for j in 1:3
        println("q[$i, $j] = ", value(q[i, j]))
    end
end

#Exibindo as sobras de cada usina
println("Sobra na usina 1: ", 6 -  value(sum(q[1, j] for j in 1:3)) )
println("Sobra na usina 2: ", 6 -  value(sum(q[2, j] for j in 1:3)) )



Custo mínimo de distribuição: 564.0
Quantidades ótimas de distribuição:
q[1, 1] = 3.0
q[1, 2] = 0.0
q[1, 3] = 3.0
q[2, 1] = 2.0
q[2, 2] = 4.0
q[2, 3] = 0.0
Sobra na usina 1: 0.0
Sobra na usina 2: 0.0


# Problema 1.4

In [68]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, S >= 0, Int)
@variable(model, C >= 0, Int)

# Função objetivo
@objective(model, Max, 5S + 4C)

# Restrições de limitação de matéria-prima
@constraint(model, 2S+C <= 78)

# Restrições de horas trabalhadas
@constraint(model, 10S+12C == 600)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model))
println("Quantidades ótimas de produção:")
println("S = ", value(S))
println("C = ", value(C))
#Horas trabalhadas 
println("Horas trabalhadas em sapatos: ", 10*value(S) )
println("Horas trabalhadas em cintos: ", 12*value(C) )


Lucro máximo: 240.0
Quantidades ótimas de produção:
S = 24.0
C = 30.0
Horas trabalhadas em sapatos: 240.0
Horas trabalhadas em cintos: 360.0


# Problema 1.5

In [70]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, QA >= 0, Int)
@variable(model, QB >= 0, Int)

# Função objetivo
@objective(model, Max, 20QA + 30QB)

# Restrições de limitação de matéria-prima
@constraint(model, (1/4)*QA <= 500)
@constraint(model, (1/4)*QA + (1/2)*QB<= 200)
@constraint(model, (1/2)*QA + (1/2)*QB<= 200)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model))
println("Quantidades ótimas de produção:")
println("QA = ", value(QA))
println("QB = ", value(QB))

#Exibindo o excedente de matéria-prima
println("Excedente de A: ", 500 -  (1/4)*value(QA) )
println("Excedente de B: ", 200 -  (1/4)*value(QA) - (1/2)*value(QB) )
println("Excedente de C: ", 200 -  (1/2)*value(QA) - (1/2)*value(QB) )

Lucro máximo: 12000.0
Quantidades ótimas de produção:
QA = 0.0
QB = 400.0
Excedente de A: 500.0
Excedente de B: 0.0
Excedente de C: 0.0


# Problema 1.6


In [89]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, q[1:4, 1:3], Int)

# Função objetivo
c = [5.5, 4.5, 3.5]  # Venda dos diesel 1,2,3
d = [3, 6, 4, 1] # Custo dos petróleos 1,2,3,4
@objective(model, Max, sum((q[i,j] * c[j]) for i in 1:4, j in 1:3) - sum((q[i,j] * d[i]) for i in 1:4, j in 1:3))

# Restrições de composição
# Diesel A
@constraint(model, (0.3) * sum(q[i,1] for i in 1:4) >= q[1,1])
@constraint(model, (0.4) * sum(q[i,1] for i in 1:4) <= q[2,1])
@constraint(model, (0.5) * sum(q[i,1] for i in 1:4) >= q[3,1])
# Diesel B
@constraint(model, (0.5) * sum(q[i,2] for i in 1:4) >= q[1,2])
@constraint(model, (0.1) * sum(q[i,2] for i in 1:4) >= q[2,2])
# Diesel C
@constraint(model, (0.7) * sum(q[i,3] for i in 1:4) >= q[1,3])

# Restrições de matéria-prima
@constraint(model, sum(q[1,j] for j in 1:3) <= 3000)
@constraint(model, sum(q[2,j] for j in 1:3) <= 2000)
@constraint(model, sum(q[3,j] for j in 1:3) <= 4000)
@constraint(model, sum(q[4,j] for j in 1:3) <= 1000)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model))

# Quantidades ótimas de produção de cada diesel
for j in 1:3
    println("Quantidade ótima de diesel tipo $j: ", sum(value(q[i,j]) for i in 1:4))
end

# Excedentes de matéria-prima
println("Excedente de matéria-prima:")
println("Excedente de 1: ", 3000 - sum(value(q[1,j]) for j in 1:3))
println("Excedente de 2: ", 2000 - sum(value(q[2,j]) for j in 1:3))
println("Excedente de 3: ", 4000 - sum(value(q[3,j]) for j in 1:3))
println("Excedente de 4: ", 1000 - sum(value(q[4,j]) for j in 1:3))


MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.